# Regex match and search

Weronika Ormaniec

Do wyszukiwania wyrażeń regularnych wykorzystano niedeterministyczny automat skończony i równoległe przeszukiwanie jego stanów zgodnie z algorytmem Thompsona. Zaimplementowano funkcje *match* i *search*. *match* sprawdza, czy dane słowo spełnia wyrażenie regularne natomiast *search* znajduje końce wystąpień wyrażenia regularnego w dłuższym tekście.

In [1]:
import string 

In [4]:
class State:
    def __init__(self, c_=-1, s_ = None, out_ = None, out1_ = None, lastlist_ = None):
        self.c = c_
        self.s = s_
        self.out = out_ 
        self.out1 = out1_ 
        self.lastlist = lastlist_
        self.parent = None
        self.visited = True
        
    def __repr__(self):
        if self.c==0:
            return f'|{self.s}, {id(self)}, {id(self.out)}, {id(self.out1)}|'
        else:
            return f'|{self.c}, {id(self)}, {id(self.out)}, {id(self.out1)}|'

class NFA_fragment:
    def __init__(self, start_=None, out_=[]):
        self.start = start_
        self.out = out_
        
    def __repr__(self):
        return f'start: {self.start}, out: {self.out}'
        
def patch(list_of_ends, node):
    for end_pointer in list_of_ends:
        if end_pointer.parent.out is end_pointer:
            end_pointer.parent.out = node
        elif end_pointer.parent.out1 is end_pointer:
            end_pointer.parent.out1 = node
    return list_of_ends
        
def postfix_to_NFA(postfix):
    stack = []
    alphabet = string.ascii_letters + string.digits + string.whitespace + '.'
    
    i = 0
    while i <len(postfix):
        letter = postfix[i]
        if letter in alphabet:
            s = State(c_=0, s_=letter, out_=State(), out1_=State())
            s.out.parent = s
            s.out1.parent = s
            stack.append(NFA_fragment(start_=s, out_=[s.out]))
        elif letter == '?':
            e = stack.pop()
            s = State(c_='Split', out_=e.start, out1_=State())
            s.out.parent = s
            s.out1.parent = s
            stack.append(NFA_fragment(start_=s, out_=e.out+[s.out1]))
        elif letter == '*':
            e = stack.pop()
            s = State(c_='Split', out_=e.start, out1_=State())
            s.out.parent = s
            s.out1.parent = s
            patch(e.out, s)
            stack.append(NFA_fragment(start_=s, out_=[s.out1]))
        elif letter == '+':
            e = stack.pop()
            s = State(c_='Split', out_=e.start, out1_=State())
            s.out.parent = s
            s.out1.parent = s
            patch(e.out, s)
            stack.append(NFA_fragment(start_=e.start, out_=[s.out1]))
        elif letter == '_':
            e2 = stack.pop()
            e1 = stack.pop()
            e1.out = patch(e1.out, e2.start)
            stack.append(NFA_fragment(start_=e1.start, out_=e2.out))
        elif letter == '|':
            e2 = stack.pop()
            e1 = stack.pop()
            s = State(c_='Split', out_=e1.start, out1_=e2.start)
            stack.append(NFA_fragment(start_=s, out_=e1.out+e2.out))
        elif letter == '[':
            category = ""
            i += 1
            while postfix[i] != ']':
                category += postfix[i]
                i+=1
            s = State(c_=0, s_=category, out_=State(), out1_=State())
            s.out.parent = s
            s.out1.parent = s
            stack.append(NFA_fragment(start_=s, out_=[s.out]))
        elif letter == '\\':
            i += 1
            if postfix[i] == 's':
                category = string.ascii_letters
            elif postfix[i] == 'd':
                category = string.digits
            elif postfix[i] == 'w':
                category = string.whitespace
            s = State(c_=0, s_=category, out_=State(), out1_=State())
            s.out.parent = s
            s.out1.parent = s
            stack.append(NFA_fragment(start_=s, out_=[s.out]))
        i += 1

    e = stack.pop()
    matchstate = State(c_='Match')
    
    patch(e.out, matchstate)
    return e.start

def ismatch(some_list):
    for state in some_list:
        if state.c == 'Match':
            return True
    return False

def dfs_set(node, condition):
    if node is not None and node.visited == condition:
        node.lastlist = None
        node.visited = not condition
        dfs_set(node.out, condition)
        dfs_set(node.out1, condition)
    
def match(start, text):
    dfs_set(start, start.visited)
    clist = []
    nlist = []
    listid = 1
    clist = add_state(clist, start, listid)
    
    for letter in text:
        nlist, listid = step(clist, letter, nlist, listid)
        clist, nlist = nlist, clist
        if len(clist)==0:
            return False
        
    return ismatch(clist)

def add_state(some_list, state, listid):
    if state.c == 'Split':
        if state.out.lastlist != listid:
            state.out.lastlist = listid
            some_list.append(state.out)
            
        if state.out1.lastlist != listid:
            state.out1.lastlist = listid
            some_list.append(state.out1)
    else:
        if state.lastlist != listid:
            state.lastlist = listid
            some_list.append(state)
    return some_list
    
    
def step(clist, letter, nlist, listid):
    listid+=1
    nlist = []
    for state in clist:
        if state.s is not None and (letter in state.s or state.s == '.'):
            nlist = add_state(nlist, state.out, listid)
            
    return nlist, listid

def infix_to_postfix(regex):
    alphabet = string.ascii_letters + string.digits + string.whitespace + '.'
    concat_added = ""
    i = 0
    in_brackets = False
    while i < len(regex):
        letter = regex[i]
        if letter == '[': 
            in_brackets = not in_brackets
            if len(concat_added)>0:
                concat_added += '_'
        elif letter == ']':
            in_brackets = not in_brackets
        elif letter == '\\':
            if len(concat_added)>0:
                concat_added += '_'
            concat_added += regex[i:i+2]
            i += 2
            continue
        elif concat_added == "":
            concat_added = letter
            i+= 1
            continue
        elif (not in_brackets and 
        ((letter in alphabet and (concat_added[-1] in alphabet or concat_added[-1] in ['*', '+', '?', ']'])) or
                                  (letter in ['('] and concat_added[-1] in alphabet))):
            concat_added += '_'
        concat_added += letter
        i += 1
    regex = concat_added
    
    stack = []
    postfix = ""
    precedense = {'|':1, '_':2, '?':3, '+':3, '*':3}
    i = 0
    while i < len(regex):
        letter = regex[i]
        if letter in alphabet:
            postfix += letter
        elif letter == ')':
            while stack[-1] != '(':
                operand = stack.pop()
                postfix += operand
            stack.pop()
        elif letter == '(':
            stack.append('(')
        elif letter in precedense.keys():
            while len(stack)>0 and stack[-1]!='(' and precedense[stack[-1]]>=precedense[letter]:
                operand = stack.pop()
                postfix += operand
            stack.append(letter)
        elif letter=='[':
            postfix += letter
            i += 1
            while regex[i] != ']':
                postfix += regex[i]
                i += 1
            postfix += regex[i]
        elif letter == '\\':
            postfix += regex[i:i+2]
            i += 1
        i += 1
            
    while len(stack)>0:
        operand = stack.pop()
        postfix += operand

    return postfix

def regex_to_NFA(regex):
    postfix = infix_to_postfix(regex)
    return postfix_to_NFA(postfix)
            
def add_state(some_list, state, listid):
    if state.c == 'Split':
        if state.out.lastlist != listid:
            state.out.lastlist = listid
            some_list.append(state.out)
            
        if state.out1.lastlist != listid:
            state.out1.lastlist = listid
            some_list.append(state.out1)
        
    else:
        if state.lastlist != listid:
            state.lastlist = listid
            some_list.append(state)
           
    return some_list

            
def search_step(clist, letter, nlist, listid, output, start):
    listid+=1
    nlist = []
    if start.c == 'Split' or (start.s is not None and (letter in start.s or start.s == '.')):
        clist = add_state(clist, start, listid)
    for state in clist:
        if state.s is not None and (letter in state.s or state.s == '.'):
            nlist = add_state(nlist, state.out, listid) 
            if state.out.c == 'Match' or (state.out.c == 'Split' and (state.out.out.c == 'Match' or
                                                                      state.out.out1.c == 'Match')):
                
                output.append(listid-1)

    return nlist, listid, output
    
def search(start,text):
    dfs_set(start, start.visited)
    clist = []
    nlist = []
    listid = 0
    output = []
    
    for letter in text:
        nlist, listid, output = search_step(clist, letter, nlist, listid, output, start)
        clist, nlist = nlist, clist
        
    return output

Prosty wzorzec bez znaków specjalnych.

In [5]:
NFA = regex_to_NFA('algorytmy tekstowe')
print(f'Match: {match(NFA, "algorytmy tekstowe")}')
print(f'Match: {match(NFA, "algorytmy")}')
text = "algorytmy tekstowe sa fajne"
found = search(NFA, text)
print(f'End of regex found at: {found}')
for f in found:
      print(text[:f+1])

Match: True
Match: False
End of regex found at: [17]
algorytmy tekstowe


Wzorzec z '.' zamiast dowolnego znaku.

In [6]:
NFA = regex_to_NFA('algorytmy teksto.e')
print(f'Match: {match(NFA, "algorytmy tekstome")}')
print(f'Match: {match(NFA, "algorytmy")}')
text = "algorytmy tekstoze sa fajne"
found = search(NFA, text)
print(f'End of regex found at: {found}')
for f in found:
      print(text[:f+1])

Match: True
Match: False
End of regex found at: [17]
algorytmy tekstoze


Operatory '*', '+', '?'.

In [7]:
NFA = regex_to_NFA('a*lgorytmy? teksto+we')
print(f'Match: {match(NFA, "aaaalgorytmy tekstooowe")}')
print(f'Match: {match(NFA, "lgorytm tekstowe")}')
print(f'Match: {match(NFA, "lgorytm tekstwe")}')
text = "aaaalgorytmy tekstooowe sa fajne ale lgorytm tekstowe bywaja trudne"
found = search(NFA, text)
print(f'End of regex found at: {found}')
for f in found:
      print(text[:f+1])

Match: True
Match: True
Match: False
End of regex found at: [22, 52]
aaaalgorytmy tekstooowe
aaaalgorytmy tekstooowe sa fajne ale lgorytm tekstowe


Wzorce z nawiasami.

In [8]:
NFA = regex_to_NFA('(a(lg)+o)*rytmy tekst(ow)?e')
print(f'Match: {match(NFA, "alglgoalgorytmy tekstowe")}')
print(f'Match: {match(NFA, "rytmy tekste")}')
print(f'Match: {match(NFA, "lgorytm tekstwe")}')
text = "alglgoalgorytmy tekstowe sa fajne ale rytmy tekste bywaja trudne"
found = search(NFA, text)
print(f'End of regex found at: {found}')
for f in found:
      print(text[:f+1])

Match: True
Match: True
Match: False
End of regex found at: [23, 49]
alglgoalgorytmy tekstowe
alglgoalgorytmy tekstowe sa fajne ale rytmy tekste


Grupy znaków w '[]' i klasy znaków.

In [9]:
NFA = regex_to_NFA('algo\sytmy\wtekst[owe3]*we\d')
print(f'Match: {match(NFA, "algorytmy tekstowe1")}')
print(f'Match: {match(NFA, "algorytmy tekstoewwe1")}')
print(f'Match: {match(NFA, "algo4ytmy tekstowe4")}')
text = "algorytmy tekstowe1 sa fajne ale algorytmy tekstoewwe1 bywaja trudne"
found = search(NFA, text)
print(f'End of regex found at: {found}')
for f in found:
      print(text[:f+1])

Match: True
Match: True
Match: False
End of regex found at: [18, 53]
algorytmy tekstowe1
algorytmy tekstowe1 sa fajne ale algorytmy tekstoewwe1
